## 问题分类算法

### 一、问题描述

知识图谱的自动问答系统，问题分类属于中文短文本分类，问题描述如下：
输入一个中文问句，把问句分成下面四种类型：
1）疫情
2）实体
3）知识
4）未知

1）疫情：是指某实体（国家、地区、省市、机构）的疫情情况（确诊、治愈、死亡人数），或者防疫措施、政策等。
广元的疫情情况如何？ 
美国疫情情况如何？
四川有哪些防控措施？
四川信息职业技术学院有哪些防控措施？
四川信息职业技术学院的防疫政策？

2）实体：是指查询某实体（国家、地区、省市、机构、人）的情况
钟南山是谁？
美国是什么？
尼日利亚是哪里？

3）知识：特指新冠肺炎相关的知识，其他知识不管。所以主体包括（新冠、新型冠状、新型肺炎等等）。
新冠肺炎来源？
新冠肺炎是什么？
怎么预防新型冠状肺炎？
哪些药物可以治新冠肺炎？
新冠肺炎有特效药吗？
新冠肺炎有什么药？
新冠病毒有疫苗吗？

4）未知：无法分类的问题都归入到未知
今天星期几？
刘德华是谁？

### 二、经典机器学习方法以及深度学习方法

经典的机器学习方法采用获取tf-idf文本特征，分别喂入logistic regression分类器和随机森林分类器的思路，并对两种方法做性能对比。
基于深度学习的文本分类，这里主要使用卷积神经网络以及循环神经网络进行中文文本分类。
CNN+RNN模型：
输入层：汉字Embedding，填充为不超过32个汉字。
CNN层：

### 三、数据集准备

数据集比较困难，现在想到的办法是从微博关键词搜索结果中找到所有疑问句，然后对疑问句进行人工打label，再进行数据增强，获得训练集和测试集。

1）生成疑问句：先用pyltp完成分句操作，然后查找问号结尾的句子。对句子进行数据清洗（去掉特殊符号）

2）人工标注：将所有疑问句放到excel表格中，进行人工标注。大概需要标注500-1000个，作为种子问题。

3）数据增强：使用neo4j数据库中的实体和关系，对疑问句中的人名、地名、政府机构、国家、城市等进行随机替换，基本可以生成任意多个问题。

In [6]:
import os
import re
from pyltp import SentenceSplitter # 中文分句

def clean_text(txt):
    #txt = re.sub('[0-9a-zA-Z]+', '', txt)
    txt = txt.replace('\n','')
    txt = txt.replace('\t','')
    txt = txt.replace('【','')
    txt = txt.replace('】','')
    txt = txt.replace('/','')
    txt = txt.replace('\\','')
    txt = txt.replace('?','？')
    txt = txt.replace(',','，')
    txt = txt.replace(';','；')
    txt = txt.replace("'",'')
    txt = txt.replace('"','')
    txt = txt.replace(' ','')
    txt = txt.replace('　','')
    return txt

def find_questions_in_file( txt_file ):
    with open(txt_file,'r',encoding='utf8') as f:
        txt = f.read()
        txt = clean_text(txt)
        sents = SentenceSplitter.split(txt)# 分句
        #print(len(sents))
        qlist = list()
        for s in sents:
            if s[len(s)-1]=='？' and len(s)>3 and len(s)<=32:
                qlist.append(s+'\n')
    return qlist

def save_questions( qlist, outfile ):
    with open(outfile,'w',encoding='utf8') as f:
        f.writelines(qlist)

rootdir = './dataset/weibo'
all_files = os.listdir(rootdir) #列出文件夹下所有的目录与文件
questions = list()
for i in range(0,len(all_files)):
    path = os.path.join(rootdir,all_files[i])
    if os.path.isfile(path):
        questions += find_questions_in_file(path)

save_questions(questions,'./dataset/questions.txt')
print(len(questions))

11473


In [7]:
#再对questions.txt进行二次清洗
import csv

KEY_WORDS=list(['ECMO','措施','政策','策略','监管','案','例','诊','院','治','愈','死','去世','逝世',
    '多少','人','是','哪位','什么','哪里','怎么','新冠','新型冠状','来源','药物','特效药',
    '治疗','症','预','护','设备','非典','SARS','MERS','CDC','病毒','炎','复工','健康码','封城','感染',
    '传染','呼吸','毒','传播','检疫','H1N1','医','药','病','疾','疫','冠','肺','省','市','国'])
lines = list()
qdict = dict()
with open('./dataset/questions.txt','r',encoding='utf8') as f:
    lines = f.readlines()
    out = list()
    for ll in lines:
        for w in KEY_WORDS:
            if ll.find(w)>=0:
                if ll in qdict:
                    continue
                else: 
                    qdict[ll] = 1
                    out.append(ll)#+'\n')
save_questions( out, './dataset/q2.txt')


加载nodes和relations文件，对问句中的人名、地名、机构名、名词做替换，每个问句随机替换10次

In [4]:
#import pandas as pd
import jieba.posseg as pg
import csv
import random

entity_file={'nr':'人名.csv','ns':'地区.csv','nt':'机构.csv','nz':'医学.csv'}
root_path = './dataset/'
entity_dict = dict()

def load_stopwords( stopfile ):
    stopword_dict = dict()
    with open(stopfile,'r',encoding='utf8') as f:
        lines = f.readlines()
        for l in lines:
            l = l.replace('\n','')
            stopword_dict[l] = 1
    return stopword_dict

def load_entity():
    for k, efile in entity_file.items():
        entity_dict[k] = list()
        with open(root_path+efile,'r',encoding='utf8') as f:
            lines = f.readlines()
            for l in lines:
                l = l.split(',')
                entity_dict[k].append(l[0])

def change_entity( id, numbers ): #随机读取n个数字
    entity=list()
    for i in range(numbers):
        index = random.randint(0,len(entity_dict[id])-1)
        entity.append(entity_dict[id][index])
    return entity

def check_jieba():
    with open('./dataset/q3.txt','r',encoding='utf8') as f:
        csvfile = csv.reader(f)
        lines = list()
        for q in csvfile:
            wordpos = pg.cut(q[0])
            line = ''
            for w in wordpos:
                lines += w.word + '(' + w.flag + ') '
            line += '\n'
            lines.append(line)
    with open('./dataset/check.txt','w',encoding='utf8') as f:
        f.writelines(lines)

def data_augment( stopword_dict ):
    expand_questions = dict()
    with open('./data/q3.txt','r',encoding='utf8') as f:
        csvfile = csv.reader(f)
        for q in csvfile:
            if len(q)==2:
                expand_questions[q[0]] = q[1] #先增加原句
                wordpos = pg.cut(q[0])
                expand = None
                wordlist=list()
                for w in wordpos: #我 r,爱 v,北京 ns,天安门 ns，钟南山 ns
                    wordlist.append(w)
                
                if len(wordlist)>6:
                    continue

                for i in range(len(wordlist)):
                    if wordlist[i].word in stopword_dict:
                        continue
                    if wordlist[i].flag in entity_file:
                        changes = change_entity(wordlist[i].flag, 200)
                        for c in changes:
                            ques = ''
                            for j in range(len(wordlist)):
                                if i==j:
                                    ques += c
                                else:
                                    ques += wordlist[j].word
                            if len(ques)<=20:
                                expand_questions[ques]=q[1]

    questions = list()
    for k,v in expand_questions.items():
        line = k+','+v+'\n'
        questions.append(line)
    random.shuffle(questions) #随机乱序
    
    n = len(questions)
    ntrain = int(n*0.8)
    nval = int(n*0.1)
    with open('./data/train.txt','w',encoding='utf8') as f:
        f.writelines(questions[:ntrain])
    with open('./data/val.txt','w',encoding='utf8') as f:
        f.writelines(questions[ntrain:ntrain+nval])
    with open('./data/test.txt','w',encoding='utf8') as f:
        f.writelines(questions[ntrain+nval:])
    print(n,ntrain,nval)

stopword_dict = load_stopwords('./dataset/stopwords.txt')
load_entity()
#check_jieba()
data_augment(stopword_dict)

18753 15002 1875
